In [ ]:
from models_impute import *
from pygrinder import block_missing

In [ ]:
data = pd.read_csv("./data/lorenz/lorenz_dataset_0_timeseries.csv",header=None)
data = data.values

In [ ]:
cg = pd.read_csv('./causality_matrices/lorenz_causality_matrix.csv', header=None)
cg = cg.values
model_params = {
        'num_levels': 10,
        'kernel_size': 8,
        'dilation_c': 2
    }

In [ ]:
data = block_missing(data[np.newaxis,...],factor=0.1, block_len=3, block_width=3)
data = data[0]
data

In [ ]:
data_imputed = impute(data, cg, model_params)
data_imputed

In [ ]:
# test_miracle.py
import numpy as np
from baseline import miracle_impu

# 创建测试数据
test_data = np.random.randn(100, 10)
test_data[np.random.random((100, 10)) < 0.1] = np.nan

print("测试数据形状:", test_data.shape)
print("缺失值数量:", np.isnan(test_data).sum())

try:
    result = miracle_impu(test_data)
    print("MIRACLE结果形状:", result.shape if result is not None else "None")
    print("MIRACLE结果类型:", type(result))
    
    if result is not None:
        print("填补后缺失值:", np.isnan(result).sum())
    else:
        print("❌ MIRACLE返回了None")
        
except Exception as e:
    print(f"❌ MIRACLE测试失败: {e}")
    import traceback
    traceback.print_exc()

In [34]:
import pandas as pd
import numpy as np

# 读取 CSV 文件
zero_matrix = pd.read_csv("zero_impu_matrix.csv").values  # shape: (T, N)

# 构造掩码矩阵：0 的位置为 0，其余为 1
mask_matrix = np.where(zero_matrix == 0, 0, 1)

# 保存新矩阵到 CSV
pd.DataFrame(mask_matrix).to_csv("zero_impu_mask.csv", index=False)


In [1]:
import numpy as np
import pandas as pd

In [35]:
pred = pd.read_csv('./train_result.csv').values
gt = pd.read_csv('./gt_matrix.csv').values

In [36]:
import torch.nn.functional as F
import torch

In [40]:
mask_matrix = 1 - mask_matrix

In [41]:
pred*mask_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.6950956 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.57569295, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.8215528 , ..., 0.        , 0.        ,
        0.        ]])

In [42]:
gt*mask_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        , 43.24953015, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 39.36363454, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 46.24124722, ...,  0.        ,
         0.        ,  0.        ]])

In [43]:
res = F.mse_loss(torch.tensor(pred*mask_matrix), torch.tensor(gt*mask_matrix))
res

tensor(2357.7471, dtype=torch.float64)

In [8]:
import tempfile
import os

tmp_file = tempfile.NamedTemporaryFile(delete=False)
tmp_file.close()
os.remove(tmp_file.name)


In [10]:
import os
import shutil

# 原始目录和目标目录路径
src_dir = './data/III'        # 替换为你的源目录路径
dst_dir = './data/mimic-iii'   # 替换为你的目标目录路径

# 创建目标目录（如果不存在）
os.makedirs(dst_dir, exist_ok=True)

# 获取源目录中的所有文件名（按名称排序，可改为按修改时间等）
all_files = sorted(f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f)))

# 选择前100个文件
files_to_move = all_files[:100]

# 移动文件
for fname in files_to_move:
    shutil.move(os.path.join(src_dir, fname), os.path.join(dst_dir, fname))

print(f"已成功移动 {len(files_to_move)} 个文件到 {dst_dir}")


已成功移动 100 个文件到 ./data/mimic-iii


In [2]:
import os
import pandas as pd
import numpy as np
from baseline import *
# 指定目录
data_dir = "./data/III"  # 替换为你的目录路径

# 获取目录下第一个 CSV 文件
csv_files = [f for f in os.listdir(data_dir) if f.endswith(".csv")]
assert csv_files, "目录中没有找到 CSV 文件"
csv_path = os.path.join(data_dir, csv_files[0])

# 读取数据
df = pd.read_csv(csv_path)
mx = df.values.astype(np.float32)

# 记录原始缺失位置
nan_pos = np.argwhere(np.isnan(mx))

# 填补缺失值
imputed_mx = tsde_impu(mx)

# 打印填补前后的值（仅缺失位置）
print(f"填补前后对比（仅缺失位置）：{imputed_mx}")



🚀 使用GPU: cuda
🔄 使用随机初始化模型
🔄 开始生成 20 个样本...


生成样本:   0%|          | 0/20 [00:00<?, ?it/s]

✅ 填补完成!
填补前后对比（仅缺失位置）：[[ 9.30000000e+01  1.37000000e+02  6.40000000e+01 ...  5.98659515e-01
  -2.10130736e-02 -1.87176481e-01]
 [ 3.68497670e-01  6.99941069e-03 -4.86000597e-01 ... -1.38138860e-01
  -1.94688439e-01  7.00000000e+00]
 [ 4.47865754e-01 -7.76937246e-01 -8.15001037e-03 ...  3.61706853e-01
   7.32253551e-01  1.84236705e-01]
 ...
 [ 3.10085732e-02 -2.25222800e-02  2.42479473e-01 ...  7.31340766e-01
   7.27256894e-01  2.74206340e-01]
 [-4.31338668e-01 -1.99973568e-01 -4.83057171e-01 ...  1.24743894e-01
   2.11212799e-01  3.76310199e-01]
 [-3.97869200e-01 -2.00192869e-01 -5.10577917e-01 ...  4.56133872e-01
  -4.72238392e-01 -3.60585958e-01]]


In [1]:
import pkgutil
import sys

for _, name, _ in pkgutil.iter_modules():
    if 'tsde' in name:
        print(name)
